In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
data_path = 'procDF2.csv'
processed_df = pd.read_csv(data_path)
processed_df

,DIS,FLO,LOY,PRI,PIM,FEA,LAG,CHO
0,-0.43600,-0.056,1.0,0.1429,0.03510,-0.175710,1.0,9
1,3.85290,-0.576,1.0,0.8136,-0.48915,-0.588930,1.0,9
2,4.58810,0.260,1.0,0.1502,-0.09050,-0.195670,1.0,9
3,61.55460,-0.139,1.0,0.0817,-0.08200,-0.435940,1.0,9
4,1.70100,0.259,1.0,-0.1534,-0.05960,0.015470,1.0,9
...,...,...,...,...,...,...,...,...
3146269,4.10573,0.135,0.0,-2.1534,0.08141,5.524900,1.0,5
3146270,37.22023,2.173,0.0,-2.5829,0.11651,0.619060,1.0,5
3146271,0.03146,0.348,0.0,-1.2892,-0.19316,0.032053,1.0,5
3146272,5.57263,0.168,0.0,-2.1721,0.12531,1.746300,1.0,5


In [3]:
shops_features = ['DIS','FLO','LOY','PRI','PIM','FEA','LAG']
X = processed_df[shops_features]
y = processed_df.CHO


transformer_num = make_pipeline(
    StandardScaler()
)


X_train, X_valid, y_train, y_valid = \
    train_test_split(X, y,stratify=y, test_size=0.33)



X_train = transformer_num.fit_transform(X_train)
X_valid = transformer_num.transform(X_valid)

Untuned model definition

In [4]:
default_model = LogisticRegression(random_state=1)
print("Default parameters in model: \n", default_model.get_params())

Default parameters in model: 
 {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 1, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [8]:
param_grid = {
    "C":[100, 10, 1.0, 0.1, 0.01],
    "solver":['lbfgs','newton-cg', 'sag', 'saga']
}
param_grid

{'C': [100, 10, 1.0, 0.1, 0.01],
 'solver': ['lbfgs', 'newton-cg', 'sag', 'saga']}

In [9]:
opt = BayesSearchCV(
    LogisticRegression(random_state=1),
    param_grid,
    n_iter=30,
    n_jobs=-1,
    cv = StratifiedKFold(n_splits=3, shuffle=True),
    random_state=1,
    verbose=0
)

opt.fit(X_train, y_train)

E:\Programy\Python\Python38\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
E:\Programy\Python\Python38\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
E:\Programy\Python\Python38\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
E:\Programy\Python\Python38\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
E:\Programy\Python\Python38\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


BayesSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
              estimator=LogisticRegression(random_state=1), n_iter=30,
              n_jobs=-1, random_state=1,
              search_spaces={'C': [100, 10, 1.0, 0.1, 0.01],
                             'solver': ['lbfgs', 'newton-cg', 'sag', 'saga']})

In [10]:
opt.best_params_

OrderedDict([('C', 100.0), ('solver', 'newton-cg')])

In [11]:
opt_best = opt.best_estimator_
print("Best parameters in random: \n", opt_best)
opt_accuracy = cross_val_score(opt_best, X_train, y_train, cv=StratifiedKFold(n_splits=3, shuffle=True))
default_accuracy = cross_val_score(default_model, X_train, y_train, cv=StratifiedKFold(n_splits=3, shuffle=True))

Best parameters in random: 
 LogisticRegression(C=100.0, random_state=1, solver='newton-cg')


E:\Programy\Python\Python38\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
E:\Programy\Python\Python38\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
E:\Programy\Python\Python38\lib\site-packages\scipy\optimize\linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
E:\Programy\Python\Python38\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
E:\Programy\Python\Python38\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


RandomSearchCV accuracy:  [0.50985245 0.50934865 0.50967386]
Default model accuracy:  [0.50968452 0.51013281 0.50914445]


In [14]:
print("BayesSearchCV accuracy: ", opt_accuracy.mean())
print("Default model accuracy: ", default_accuracy.mean())

BayesSearchCV accuracy:  0.5096302044723918
Default model accuracy:  0.5096539234007361
